# RAG FAQ-app
 

### Step 1: Load the Data

In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader('dockuno.csv')
dataset = loader.load()

### Step 2: Load embeddings

In [ ]:
# from langchain.embeddings.openai import OpenAIEmbeddings
# myembedmodels = OpenAIEmbeddings(openai_api_key="")

from langchain_community.embeddings import OllamaEmbeddings
myembedmodels = OllamaEmbeddings(model="all-minilm")

### Step 3: store embeddings into Vector Database

In [15]:
# pip install -qU langchain-community faiss-cpu
from langchain.vectorstores import FAISS
db = FAISS.from_documents( documents=dataset , embedding=myembedmodels)


In [16]:
# db.similarity_search("what is the fees in dollars?")
def retrieve_data(query):
    similar_response = db.similarity_search(query)
    ret_data = [ doc.page_content for doc in similar_response]
    return ret_data

### Step 4: Prepare template prompt


In [11]:
dockuno_template = """
You are a world class business development representative for Dockuno program at eduLearn.ai .
I will share a prospect's message with you and you will give me the best answer that I should send to this prospect based on past best practices, and you will follow ALL of the rules below:
1/ Response should be very similar or even identical to the past best practies, in terms of length, ton of voice, logical arguments and other details
2/ If the best practice are irrelevant, then try to mimic the style of the best practice to prospect's message
3/ Keep the response short and crisp and to the point

Below is a message I received from the prospect:
{message}

Here is a list of best practies of how we normally respond to prospect in similar scenarios:
{best_practice}

Please write the best response that I should send to this prospect:
"""
from langchain.prompts import PromptTemplate

myprompt = PromptTemplate(template=dockuno_template, input_variables=["message", "best_practice"])

### Step 5 : prepare llm model

In [ ]:
from langchain.chat_models import ollama
myllm = ollama.ChatOllama(model="tinyllama", temperature=0)

### prepare chain

In [17]:
from langchain.chains import LLMChain

my_chain = LLMChain(llm=myllm , prompt=myprompt)
best_practice = retrieve_data("what is the fees in dollars?")
response = my_chain.run(message="I am saurabh, what is the fees of dockuno in dollar", best_practice = best_practice)

In [18]:
print(response)

Dear Saurabh,

Thank you for reaching out to us regarding your interest in our Dockuno program. We appreciate your inquiry and would like to provide you with some information about our course fee and pricing structure.

Our course fee is based on the selected package, which includes the duration of the course, the number of days, and the level of instruction. The packages are as follows:

- 1 Day Intensive Course (3 Days): 4000 INR
- Self-Paced Learners (3 Days): 2500 INR
- Live Q&A Sessions (Periodic): 2500 INR

We understand that time is valuable, and we strive to provide you with the best possible learning experience. Our courses are designed to be self-paced, allowing learners to complete them at their own convenience. We also offer live Q&A sessions where you can ask questions directly to our instructors.

We value your time and would like to assure you that we take your needs and requirements seriously. Our team of experts is dedicated to providing you with the best possible lear